In [9]:
import math
from statistics import mean
from query_processing.process_query import process_query

In [2]:
def load_queries(filename):
    """ Load test queries"""
    queries = {}
    with open(filename, 'r', encoding='utf-8') as file:
        for line in file:
            query_id, query_text = line.strip().split('\t')
            queries[query_id] = query_text
    return queries

queries = load_queries('../data/test_queries/msmarco-test2020-queries.tsv')

In [3]:
def load_qrels(filename):
    """ Load expected relevances"""
    qrels = {}
    with open(filename, 'r', encoding='utf-8') as file:
        for line in file:
            query_id, _, doc_id, relevance = line.strip().split()
            if query_id not in qrels:
                qrels[query_id] = {}
            qrels[query_id][doc_id] = int(relevance)
    return qrels

qrels = load_qrels('../data/test_queries/2020qrels-pass.txt')


In [4]:
import time

from input_output.index_io import load_document_index, load_inverted_index_binary, load_lexicon

# Load inverted index, lexicon and document index
inverted_index_start = time.time()
print("Loading the inverted index")
inverted_index = load_inverted_index_binary('../data/inverted_index_8841823.bin')
print(f"Inverted index loaded in {time.time() - inverted_index_start} s")
total_docs = len(inverted_index)  # Adjust this if needed

lexicon_start = time.time()
print("Loading lexicon")
lexicon = load_lexicon('../data/lexicon.txt')
print(f"Lexicon loaded in {time.time() - lexicon_start} s")

document_index_start = time.time()
print("Loading document index")
document_index = load_document_index("../data/document_index.txt")
print(f"Document index loaded in {time.time() - document_index_start} s")

Loading the inverted index
Inverted index loaded in 115.75539922714233 s
Loading lexicon
Lexicon loaded in 0.03753232955932617 s
Loading document index
Document index loaded in 5.401638746261597 s


In [19]:
ndcg_list = []
# Process all test queries and compute nDCG for TF-IDF and BM25
for ranking in ["tfidf","bm25"]:
    for user_type in [2,10]:
        nb_queries = 0
        elapsed_time = 0
        ndcg = []
        for qid, query in queries.items():
            if qid not in qrels:
                continue
            k = len(qrels[qid])
            querying_start = time.time()
            # Evaluation is made for 20 documents ranking
            results = process_query(query, inverted_index, lexicon, document_index, total_docs, ranking)[:20]
            elapsed_time += time.time() - querying_start
            nb_queries += 1
            results = {docid for docid,_ in results}

            # Map resulted docs with their expected relevance
            weighted_assessed_run = []
            for docid in results:
                if docid in qrels[qid]:
                    weighted_assessed_run.append(qrels[qid][docid])
                else:
                    weighted_assessed_run.append(0)

            if len(weighted_assessed_run) == 0 :
                ndcg.append(0)
                continue

            # Compute DCG
            dcg = weighted_assessed_run[0]
            count = 1
            for weight in weighted_assessed_run[1:]:
                if user_type == 2:
                    dcg += weight / math.log2(count + 1)
                else:
                    dcg += weight / math.log10(count + 1)
                count += 1

            # Compute IDCG
            expected_results = list(qrels[qid].values())
            weighted_assessed_ideal_run = sorted(expected_results)[20:]
            idcg = weighted_assessed_ideal_run[0]
            count = 1
            for weight in weighted_assessed_ideal_run[1:]:
                if user_type == 2:
                    idcg += weight / math.log2(count + 1)
                else:
                    idcg += weight / math.log10(count + 1)
                count += 1

            ndcg.append(dcg/idcg)
            
        ndcg_list.append(mean(ndcg))
        print(str(user_type) + " nDCG " + ranking + " : ",mean(ndcg))
        print("Time elapsed : ",elapsed_time)
        print("Nb of queries : ",nb_queries)
        print("Average time for each query : ",elapsed_time/nb_queries)


2 nDCG tfidf :  0.11912212034693231
Time elapsed :  66.19834518432617
Nb of queries :  54
Average time for each query :  1.2258952811912254
10 nDCG tfidf :  0.1080524083016866
Time elapsed :  66.25589299201965
Nb of queries :  54
Average time for each query :  1.2269609813336972
2 nDCG bm25 :  0.18249851524083632
Time elapsed :  108.04627966880798
Nb of queries :  54
Average time for each query :  2.0008570309038514
10 nDCG bm25 :  0.16130841665728948
Time elapsed :  105.69976377487183
Nb of queries :  54
Average time for each query :  1.9574030328679968
